In [1]:
import numpy as np
import matplotlib.pyplot as plt

import optimal_transport.apdamd.ot
from optimal_transport.greenkhorn.ot import OT as greenkhorn
from optimal_transport.apdamd.ot import OT as apdamd
from optimal_transport.tests.sample_problem import sample_gaussian_OT_exact, simple_problem
from optimal_transport.ot import cost
import timeit

In [2]:
# Create the problem
n = 1  # dim
N = 20  # number of points
Gaussian1 = (np.zeros(n), np.eye(n))
Gaussian2 = (np.ones(n), np.eye(n) + np.triu(np.ones(n)) * 0.5)
my_problem = sample_gaussian_OT_exact(N, n, (Gaussian1, Gaussian2))


In [3]:
epss = np.linspace(0.01, 0.05, 5)
timings_greenkhorn = dict()
timings_apdamd = dict()
std_timings_greenkhorn = dict()
std_timings_apdamd = dict()
transport_plans_greenkhorn = dict()
transport_plans_apdamd = dict()
costs_greenkhorn = dict()
costs_apdamd = dict()
realised_iters_greenkhorn = dict()
realised_iters_apdamd = dict()
theoretical_iters_greenkhorn = dict()
theoretical_iters_apdamd = dict()
converge_greenkhorn = dict()
converge_apdamd = dict()

ENABLE_GREENKHORN = False

n_iter = 10

# Theoretical bounds on hte number of iterations
for i, eps in enumerate(epss):
    theoretical_iters_apdamd[(n_iter, eps)] = optimal_transport.apdamd.ot.theoretical_bound_on_iter(*my_problem[1:], eps)
    theoretical_iters_greenkhorn[(n_iter, eps)] = optimal_transport.greenkhorn.ot.theoretical_bound_on_iter(*my_problem[1:], eps)


In [4]:
for i, eps in enumerate(epss):
    if ENABLE_GREENKHORN:
        transport_plans_greenkhorn[(n_iter, eps)], _ = greenkhorn(None, *my_problem[1:], eps=eps, iter_max=n_iter)
        costs_greenkhorn[(n_iter, eps)] = (cost(my_problem[1], transport_plans_greenkhorn[(n_iter, eps)]))
        timing_greenkhorn = %timeit -o -r 2 -n 1 greenkhorn( * my_problem, eps, iter_max=n_iter)
        timings_greenkhorn[(n_iter, eps)] = np.mean(timing_greenkhorn.timings)
        std_timings_greenkhorn[(n_iter, eps)] = np.std(timing_greenkhorn.timings)
        realised_iters_greenkhorn[(n_iter, eps)] = _
        converge_greenkhorn[(n_iter, eps)] = _ < min(n_iter if n_iter is not None else np.inf,
                                                     theoretical_iters_greenkhorn[(n_iter, eps)])

    transport_plans_apdamd[(n_iter, eps)], _ = apdamd(None, *my_problem[1:], eps=eps, iter_max=n_iter)
    costs_apdamd[(n_iter, eps)] = cost(my_problem[1], transport_plans_apdamd[(n_iter, eps)])
    timing_apdamd = %timeit -o -r 5 -n 1 apdamd( * my_problem, eps, iter_max=n_iter)
    timings_apdamd[(n_iter, eps)] = np.mean(timing_apdamd.timings)
    std_timings_apdamd[(n_iter, eps)] = np.std(timing_apdamd.timings)
    realised_iters_apdamd[(n_iter, eps)] = _
    converge_apdamd[(n_iter, eps)] = _ < min(n_iter if n_iter is not None else np.inf,
                                             theoretical_iters_apdamd[(n_iter, eps)])

np.savez('./run.npz', timings_greenkhorn=timings_greenkhorn, timings_apdamd=timings_apdamd,
         std_timings_greenkhorn=std_timings_greenkhorn, std_timings_apdamd=std_timings_apdamd,
         transport_plans_greenkhorn=transport_plans_greenkhorn, transport_plans_apdamd=transport_plans_apdamd,
         costs_greenkhorn=costs_greenkhorn, costs_apdamd=costs_apdamd, realised_iters_greenkhorn=realised_iters_greenkhorn,
         realised_iters_apdamd=realised_iters_apdamd, theoretical_iters_greenkhorn=theoretical_iters_greenkhorn,
         theoretical_iters_apdamd=theoretical_iters_apdamd, converge_greenkhorn=converge_greenkhorn,
         converge_apdamd=converge_apdamd, epss=epss, my_problem=my_problem)

507 ms ± 10.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
559 ms ± 71.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
502 ms ± 7.32 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
490 ms ± 3.04 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
512 ms ± 29.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (4, 20) + inhomogeneous part.

In [18]:
a = np.array(my_problem[2])
b = np.array(my_problem[3])
M = np.array(my_problem[1])

my_simple_problem = simple_problem()
a = np.array(my_simple_problem[2])
b = np.array(my_simple_problem[3])
M = np.array(my_simple_problem[1])


array([[4.99977301e-01, 2.26989344e-05],
       [2.26989344e-05, 4.99977301e-01]])

In [25]:
greenkhorn(None, *my_simple_problem[1:], eps=0.1, iter_max=10)

(Array([[5.00000000e-01, 4.54747351e-13],
        [4.54747351e-13, 5.00000000e-01]], dtype=float64),
 Array(10, dtype=int64, weak_type=True))